In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

# Generating the training data for the Heat and Diffusion Model

In [2]:
data_dir = "./LakePIAB/"
depth_steps = 25

print(os.getcwd())

/home/robert/Projects/LakePIAB/MCL/01_data_processing


In [3]:
meterological_data_df = pd.read_csv("./../../output/py_meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0
...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0


In [4]:
input_temp_df = pd.read_csv("./../../output/py_temp_total05.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total05':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total05,depth
0,2006-01-01 02:00:00,5.406383,5.334504,1
1,2006-01-01 02:00:00,5.436557,5.372093,2
2,2006-01-01 02:00:00,5.436557,5.409226,3
3,2006-01-01 02:00:00,5.468520,5.440322,4
4,2006-01-01 02:00:00,5.498996,5.440322,5
...,...,...,...,...
3065970,2019-12-29 00:00:00,3.495852,3.496487,21
3065971,2019-12-29 00:00:00,3.672066,3.672559,22
3065972,2019-12-29 00:00:00,3.848560,3.848898,23
3065973,2019-12-29 00:00:00,4.025302,4.025472,24


In [5]:
depth_list.shape

(3065975,)

In [6]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.406383,5.334504
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.372093
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.409226
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.468520,5.440322
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.498996,5.440322
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.495852,3.496487
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.672066,3.672559
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.848560,3.848898
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,4.025302,4.025472


In [7]:
buoyancy_data_df = pd.read_csv("./../../output/py_buoyancy.csv")
#Removing Nans
#buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
#buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']
buoyancy_data_df['24'] = buoyancy_data_df['23']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2006-01-01 02:00:00,-6.670263e-06,1
1,2006-01-01 02:00:00,0.000000e+00,2
2,2006-01-01 02:00:00,-7.214821e-06,3
3,2006-01-01 02:00:00,-7.021914e-06,4
4,2006-01-01 02:00:00,-2.013305e-07,5
...,...,...,...
3065970,2019-12-29 00:00:00,1.107272e-05,21
3065971,2019-12-29 00:00:00,6.156150e-06,22
3065972,2019-12-29 00:00:00,1.243659e-06,23
3065973,2019-12-29 00:00:00,-3.657192e-06,24


In [8]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.406383,5.334504,-6.670263e-06
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.372093,0.000000e+00
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.409226,-7.214821e-06
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.468520,5.440322,-7.021914e-06
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.498996,5.440322,-2.013305e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.495852,3.496487,1.107272e-05
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.672066,3.672559,6.156150e-06
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.848560,3.848898,1.243659e-06
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,4.025302,4.025472,-3.657192e-06


In [9]:
out_diffusivity_df = pd.read_csv("./../../output/py_diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2006-01-01 02:00:00,0.000548,1
1,2006-01-01 02:00:00,0.000448,2
2,2006-01-01 02:00:00,0.000323,3
3,2006-01-01 02:00:00,0.000228,4
4,2006-01-01 02:00:00,0.000160,5
...,...,...,...
3065970,2019-12-29 00:00:00,0.000491,21
3065971,2019-12-29 00:00:00,0.000435,22
3065972,2019-12-29 00:00:00,0.000387,23
3065973,2019-12-29 00:00:00,0.000343,24


In [10]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.406383,5.334504,-6.670263e-06,0.000548
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.372093,0.000000e+00,0.000448
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.409226,-7.214821e-06,0.000323
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.468520,5.440322,-7.021914e-06,0.000228
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.498996,5.440322,-2.013305e-07,0.000160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.495852,3.496487,1.107272e-05,0.000491
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.672066,3.672559,6.156150e-06,0.000435
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.848560,3.848898,1.243659e-06,0.000387
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,4.025302,4.025472,-3.657192e-06,0.000343


In [11]:
out_temp_df = pd.read_csv("./../../output/py_temp_heat01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_heat01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_heat01,depth
0,2006-01-01 02:00:00,5.278988,1
1,2006-01-01 02:00:00,5.436561,2
2,2006-01-01 02:00:00,5.436561,3
3,2006-01-01 02:00:00,5.468524,4
4,2006-01-01 02:00:00,5.498999,5
...,...,...,...
3065970,2019-12-29 00:00:00,3.495887,21
3065971,2019-12-29 00:00:00,3.672113,22
3065972,2019-12-29 00:00:00,3.848641,23
3065973,2019-12-29 00:00:00,4.025302,24


In [12]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.406383,5.334504,-6.670263e-06,0.000548,5.278988
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.372093,0.000000e+00,0.000448,5.436561
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.409226,-7.214821e-06,0.000323,5.436561
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.468520,5.440322,-7.021914e-06,0.000228,5.468524
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.498996,5.440322,-2.013305e-07,0.000160,5.498999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.495852,3.496487,1.107272e-05,0.000491,3.495887
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.672066,3.672559,6.156150e-06,0.000435,3.672113
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.848560,3.848898,1.243659e-06,0.000387,3.848641
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,4.025302,4.025472,-3.657192e-06,0.000343,4.025302


In [13]:
out_temp_df = pd.read_csv("./../../output/py_temp_diff02.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff02':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff02,depth
0,2006-01-01 02:00:00,5.278988,1
1,2006-01-01 02:00:00,5.335771,2
2,2006-01-01 02:00:00,5.425026,3
3,2006-01-01 02:00:00,5.462485,4
4,2006-01-01 02:00:00,5.486707,5
...,...,...,...
3065970,2019-12-29 00:00:00,3.496487,21
3065971,2019-12-29 00:00:00,3.672559,22
3065972,2019-12-29 00:00:00,3.848898,23
3065973,2019-12-29 00:00:00,4.025472,24


In [14]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.406383,5.334504,-6.670263e-06,0.000548,5.278988,5.278988
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.372093,0.000000e+00,0.000448,5.436561,5.335771
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.409226,-7.214821e-06,0.000323,5.436561,5.425026
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.468520,5.440322,-7.021914e-06,0.000228,5.468524,5.462485
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.498996,5.440322,-2.013305e-07,0.000160,5.498999,5.486707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.495852,3.496487,1.107272e-05,0.000491,3.495887,3.496487
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.672066,3.672559,6.156150e-06,0.000435,3.672113,3.672559
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.848560,3.848898,1.243659e-06,0.000387,3.848641,3.848898
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,4.025302,4.025472,-3.657192e-06,0.000343,4.025302,4.025472


In [15]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [16]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [17]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.406383,5.334504,-6.670263e-06,0.000548,5.278988,5.278988,1,2
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.372093,0.000000e+00,0.000448,5.436561,5.335771,1,2
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.409226,-7.214821e-06,0.000323,5.436561,5.425026,1,2
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.468520,5.440322,-7.021914e-06,0.000228,5.468524,5.462485,1,2
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.498996,5.440322,-2.013305e-07,0.000160,5.498999,5.486707,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.495852,3.496487,1.107272e-05,0.000491,3.495887,3.496487,363,0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.672066,3.672559,6.156150e-06,0.000435,3.672113,3.672559,363,0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.848560,3.848898,1.243659e-06,0.000387,3.848641,3.848898,363,0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,4.025302,4.025472,-3.657192e-06,0.000343,4.025302,4.025472,363,0


In [18]:
temp_mix03_df = pd.read_csv("./../../output/py_temp_mix03.csv")

temp_mix03_df = temp_mix03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix03':flattened_temp, 'depth':depth_list}
temp_mix03_df = pd.DataFrame(data=data)

temp_mix03_df

,time,temp_mix03,depth
0,2006-01-01 02:00:00,5.278988,1
1,2006-01-01 02:00:00,5.335771,2
2,2006-01-01 02:00:00,5.425026,3
3,2006-01-01 02:00:00,5.462485,4
4,2006-01-01 02:00:00,5.486707,5
...,...,...,...
3065970,2019-12-29 00:00:00,3.496487,21
3065971,2019-12-29 00:00:00,3.672559,22
3065972,2019-12-29 00:00:00,3.848898,23
3065973,2019-12-29 00:00:00,4.025472,24


In [19]:
final_df = final_df.merge(temp_mix03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.406383,5.334504,-6.670263e-06,0.000548,5.278988,5.278988,1,2,5.278988
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.372093,0.000000e+00,0.000448,5.436561,5.335771,1,2,5.335771
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.436557,5.409226,-7.214821e-06,0.000323,5.436561,5.425026,1,2,5.425026
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.468520,5.440322,-7.021914e-06,0.000228,5.468524,5.462485,1,2,5.462485
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,36000000.0,5.498996,5.440322,-2.013305e-07,0.000160,5.498999,5.486707,1,2,5.486707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.495852,3.496487,1.107272e-05,0.000491,3.495887,3.496487,363,0,3.496487
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.672066,3.672559,6.156150e-06,0.000435,3.672113,3.672559,363,0,3.672559
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,3.848560,3.848898,1.243659e-06,0.000387,3.848641,3.848898,363,0,3.848898
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,36000000.0,4.025302,4.025472,-3.657192e-06,0.000343,4.025302,4.025472,363,0,4.025472


In [20]:
temp_conv03_df = pd.read_csv("./../../output/py_temp_conv04.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv04':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv04,depth
0,2006-01-01 02:00:00,5.334504,1
1,2006-01-01 02:00:00,5.372093,2
2,2006-01-01 02:00:00,5.409226,3
3,2006-01-01 02:00:00,5.440322,4
4,2006-01-01 02:00:00,5.440322,5
...,...,...,...
3065970,2019-12-29 00:00:00,3.496487,21
3065971,2019-12-29 00:00:00,3.672559,22
3065972,2019-12-29 00:00:00,3.848898,23
3065973,2019-12-29 00:00:00,4.025472,24


In [21]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.406383,5.334504,-6.670263e-06,0.000548,5.278988,5.278988,1,2,5.278988,5.334504
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.436557,5.372093,0.000000e+00,0.000448,5.436561,5.335771,1,2,5.335771,5.372093
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.436557,5.409226,-7.214821e-06,0.000323,5.436561,5.425026,1,2,5.425026,5.409226
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.468520,5.440322,-7.021914e-06,0.000228,5.468524,5.462485,1,2,5.462485,5.440322
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.498996,5.440322,-2.013305e-07,0.000160,5.498999,5.486707,1,2,5.486707,5.440322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,3.495852,3.496487,1.107272e-05,0.000491,3.495887,3.496487,363,0,3.496487,3.496487
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,3.672066,3.672559,6.156150e-06,0.000435,3.672113,3.672559,363,0,3.672559,3.672559
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,3.848560,3.848898,1.243659e-06,0.000387,3.848641,3.848898,363,0,3.848898,3.848898
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,4.025302,4.025472,-3.657192e-06,0.000343,4.025302,4.025472,363,0,4.025472,4.025472


In [22]:
temp_initial00_df = pd.read_csv("./../../output/py_temp_initial00.csv")

temp_initial00_df = temp_initial00_df[1:] # considering everything from 2nd time step

flattened_temp = temp_initial00_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_initial00_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_initial00':flattened_temp, 'depth':depth_list}
temp_initial00_df = pd.DataFrame(data=data)

temp_initial00_df

,time,temp_initial00,depth
0,2006-01-01 02:00:00,5.406383,1
1,2006-01-01 02:00:00,5.436557,2
2,2006-01-01 02:00:00,5.436557,3
3,2006-01-01 02:00:00,5.468520,4
4,2006-01-01 02:00:00,5.498996,5
...,...,...,...
3065970,2019-12-29 00:00:00,3.495852,21
3065971,2019-12-29 00:00:00,3.672066,22
3065972,2019-12-29 00:00:00,3.848560,23
3065973,2019-12-29 00:00:00,4.025302,24


In [23]:
final_df = final_df.merge(temp_initial00_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.334504,-6.670263e-06,0.000548,5.278988,5.278988,1,2,5.278988,5.334504,5.406383
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.372093,0.000000e+00,0.000448,5.436561,5.335771,1,2,5.335771,5.372093,5.436557
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.409226,-7.214821e-06,0.000323,5.436561,5.425026,1,2,5.425026,5.409226,5.436557
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.440322,-7.021914e-06,0.000228,5.468524,5.462485,1,2,5.462485,5.440322,5.468520
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,5.440322,-2.013305e-07,0.000160,5.498999,5.486707,1,2,5.486707,5.440322,5.498996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,3.496487,1.107272e-05,0.000491,3.495887,3.496487,363,0,3.496487,3.496487,3.495852
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,3.672559,6.156150e-06,0.000435,3.672113,3.672559,363,0,3.672559,3.672559,3.672066
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,3.848898,1.243659e-06,0.000387,3.848641,3.848898,363,0,3.848898,3.848898,3.848560
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,4.025472,-3.657192e-06,0.000343,4.025302,4.025472,363,0,4.025472,4.025472,4.025302


In [24]:
temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

(3065975,)
(3065975,)
(3065975,)


In [25]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,-6.670263e-06,0.000548,5.278988,5.278988,1,2,5.278988,5.334504,5.406383,-999.0
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,0.000000e+00,0.000448,5.436561,5.335771,1,2,5.335771,5.372093,5.436557,-999.0
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,-7.214821e-06,0.000323,5.436561,5.425026,1,2,5.425026,5.409226,5.436557,-999.0
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,-7.021914e-06,0.000228,5.468524,5.462485,1,2,5.462485,5.440322,5.468520,-999.0
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,-2.013305e-07,0.000160,5.498999,5.486707,1,2,5.486707,5.440322,5.498996,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,1.107272e-05,0.000491,3.495887,3.496487,363,0,3.496487,3.496487,3.495852,-999.0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,6.156150e-06,0.000435,3.672113,3.672559,363,0,3.672559,3.672559,3.672066,-999.0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,1.243659e-06,0.000387,3.848641,3.848898,363,0,3.848898,3.848898,3.848560,-999.0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,-3.657192e-06,0.000343,4.025302,4.025472,363,0,4.025472,4.025472,4.025302,-999.0


In [26]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)


time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(3065975,)
(3065975,)
(3065975,)


,time,input_obs,depth
0,2006-01-01 02:00:00,-999.0,1
1,2006-01-01 02:00:00,-999.0,2
2,2006-01-01 02:00:00,-999.0,3
3,2006-01-01 02:00:00,-999.0,4
4,2006-01-01 02:00:00,-999.0,5
...,...,...,...
3065970,2019-12-29 00:00:00,-999.0,21
3065971,2019-12-29 00:00:00,-999.0,22
3065972,2019-12-29 00:00:00,-999.0,23
3065973,2019-12-29 00:00:00,-999.0,24


In [27]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,0.000548,5.278988,5.278988,1,2,5.278988,5.334504,5.406383,-999.0,-999.0
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,0.000448,5.436561,5.335771,1,2,5.335771,5.372093,5.436557,-999.0,-999.0
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,0.000323,5.436561,5.425026,1,2,5.425026,5.409226,5.436557,-999.0,-999.0
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,0.000228,5.468524,5.462485,1,2,5.462485,5.440322,5.468520,-999.0,-999.0
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,0.000160,5.498999,5.486707,1,2,5.486707,5.440322,5.498996,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,0.000491,3.495887,3.496487,363,0,3.496487,3.496487,3.495852,-999.0,-999.0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,0.000435,3.672113,3.672559,363,0,3.672559,3.672559,3.672066,-999.0,-999.0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,0.000387,3.848641,3.848898,363,0,3.848898,3.848898,3.848560,-999.0,-999.0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,0.000343,4.025302,4.025472,363,0,4.025472,4.025472,4.025302,-999.0,-999.0


In [28]:
ice_data_df = pd.read_csv("./../../output/py_icesnow.csv")
ice_data_df = ice_data_df[1:] # considering everything from 2nd time step

num_time_steps = ice_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
ice_data_df = pd.DataFrame(np.repeat(ice_data_df.values, depth_steps, axis=0), columns=ice_data_df.columns)
ice_data_df = pd.concat([depth_df, ice_data_df], ignore_index=False, axis=1)
print(ice_data_df)

final_df = final_df.merge(ice_data_df, how='inner', on=['time','depth'])
final_df

         depth                 time       ice snow snowice
0            1  2006-01-01 02:00:00       0.0  0.0     0.0
1            2  2006-01-01 02:00:00       0.0  0.0     0.0
2            3  2006-01-01 02:00:00       0.0  0.0     0.0
3            4  2006-01-01 02:00:00       0.0  0.0     0.0
4            5  2006-01-01 02:00:00       0.0  0.0     0.0
...        ...                  ...       ...  ...     ...
3065970     21  2019-12-29 00:00:00  0.258091  0.0     0.0
3065971     22  2019-12-29 00:00:00  0.258091  0.0     0.0
3065972     23  2019-12-29 00:00:00  0.258091  0.0     0.0
3065973     24  2019-12-29 00:00:00  0.258091  0.0     0.0
3065974     25  2019-12-29 00:00:00  0.258091  0.0     0.0

[3065975 rows x 5 columns]


,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs,ice,snow,snowice
0,1,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,1,2,5.278988,5.334504,5.406383,-999.0,-999.0,0.0,0.0,0.0
1,2,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,1,2,5.335771,5.372093,5.436557,-999.0,-999.0,0.0,0.0,0.0
2,3,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,1,2,5.425026,5.409226,5.436557,-999.0,-999.0,0.0,0.0,0.0
3,4,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,1,2,5.462485,5.440322,5.468520,-999.0,-999.0,0.0,0.0,0.0
4,5,2006-01-01 02:00:00,-1.220007,590.730964,-35.085181,-41.432575,0.0,0.4,2.43924,0.013584,...,1,2,5.486707,5.440322,5.498996,-999.0,-999.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,363,0,3.496487,3.496487,3.495852,-999.0,-999.0,0.258091,0.0,0.0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,363,0,3.672559,3.672559,3.672066,-999.0,-999.0,0.258091,0.0,0.0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,363,0,3.848898,3.848898,3.848560,-999.0,-999.0,0.258091,0.0,0.0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,35.016823,49.982113,0.0,0.4,9.313737,0.115895,...,363,0,4.025472,4.025472,4.025302,-999.0,-999.0,0.258091,0.0,0.0


In [29]:
obs_array = final_df['obs_temp']
obs_array[obs_array == -999] = final_df['temp_total05']
print(obs_array)
final_df['obs_temp'] = obs_array

0          5.334504
1          5.372093
2          5.409226
3          5.440322
4          5.440322
             ...   
3065970    3.496487
3065971    3.672559
3065972    3.848898
3065973    4.025472
3065974    4.202142
Name: obs_temp, Length: 3065975, dtype: float64


/home/robert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
final_df_null = final_df.fillna('')
print(final_df_null.head)

<bound method NDFrame.head of          depth                 time  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  \
0            1  2006-01-01 02:00:00     -1.220007     590.730964   -35.085181   
1            2  2006-01-01 02:00:00     -1.220007     590.730964   -35.085181   
2            3  2006-01-01 02:00:00     -1.220007     590.730964   -35.085181   
3            4  2006-01-01 02:00:00     -1.220007     590.730964   -35.085181   
4            5  2006-01-01 02:00:00     -1.220007     590.730964   -35.085181   
...        ...                  ...           ...            ...          ...   
3065970     21  2019-12-29 00:00:00      3.860010     597.535984    35.016823   
3065971     22  2019-12-29 00:00:00      3.860010     597.535984    35.016823   
3065972     23  2019-12-29 00:00:00      3.860010     597.535984    35.016823   
3065973     24  2019-12-29 00:00:00      3.860010     597.535984    35.016823   
3065974     25  2019-12-29 00:00:00      3.860010     597.535984    35.016823  

In [31]:
final_df_null.to_csv("./../02_training/all_data_lake_modeling_in_time.csv", index=False)

In [32]:
final_df_null.to_csv("all_data_lake_modeling_in_time.csv", index=False)